# Timer

In [195]:
def print_red(text):
    print('\x1b[31m' + text + '\x1b[0m')
    
def print_yellow(text):
    print('\x1b[33m' + text + '\x1b[0m')

def print_green(text):
    print('\x1b[32m' + text + '\x1b[0m')

def print_pink(text):
    print('\x1b[35m' + text + '\x1b[0m')

def print_cyan(text):
    print('\x1b[36m' + text + '\x1b[0m')

In [196]:

matrix = np.zeros((2, len([os.path.join('../models', f) for f in os.listdir('../models') if f.endswith('.mzn')])))
matrix[0] = [0, 0, 0, 0, 0, 0, 0, 1]
print(matrix)

ValueError: could not broadcast input array from shape (8,) into shape (54,)

## Ranking all annotations

In [ ]:
import os   
import subprocess 
import random
import numpy as np

def select_winners(mnts, times):
    winners = [1] * len(mnts)
    min_mnt = min(mnts)
    min_time = min(times)
    
    print('min_mnt',min_mnt)
    print('min_time',min_time)
    
    for i, mnt in enumerate(mnts):
        if mnt == min_mnt:
            if times[i] != min_time:
                winners[i] = 0
        else:
            winners[i] = 0
        
    
    return winners

def annotations_winners(index, models_folder, instances_folder, timeout_mzn, timeout):
    annotations = [os.path.join(models_folder, f) for f in os.listdir(models_folder) if f.endswith('.mzn')]
    instance = [os.path.join(instances_folder, f) for f in os.listdir(instances_folder) if f.endswith('.dzn')][index]
    
    mnts = [0] * len(annotations)
    times = [0] * len(annotations)
    timeout_mzn = timeout_mzn*1000
    
    for i, annotation in enumerate(annotations):        
        # Run the minizinc command for the current annotation and data file  
        cmd = f"minizinc --solver COIN-BC --output-time {annotation} {instance} --output-time --solver-time-limit {timeout_mzn}" 
        proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
        try: 
            # Get output from fzn
            stdout, stderr = proc.communicate(timeout=timeout) 
            stdout = stdout.decode()
            try:
                # Store value mnt (temp).
                start = stdout.find('mnt = ')+6
                end = stdout.find(';',start)
                mnt = int(stdout[start:end])
                # Store value time (temp).
                time = float(stdout.split(' ')[-2])
            except:
                mnt = float('inf')
                time = float('inf')
            print_green(f'{annotation[10:-4]} with {instance[3:-4]} SOLVED: {mnt} in {time:.2f} seconds.')

        except subprocess.TimeoutExpired: 
            print_red(f'Error from minizinc: Stopping.')                
            mnt = float('inf')
            time = float('inf')
            if os.name == 'nt': # If the os is Windows
                subprocess.call(['taskkill', '/F', '/T', '/PID', str(proc.pid)]) # Force kill the process
            else:
                os.killpg(os.getpgid(proc.pid), signal.SIGTERM) # Kill the process group
                
        mnts[i] = mnt
        times[i] = time
    # print('mnts',mnts)
    # print('times',times)
    
    return select_winners(mnts,times)

matrix = np.zeros((20, len([os.path.join('../models', f) for f in os.listdir('../models') if f.endswith('.mzn')])))
for i in range(0,20):
    matrix[i]=np.array(annotations_winners(random.randint(0,20), '../models', '../instances',5, 10))

#r1 = np.array(annotations_winners(0, '../models', '../instances',5, 10))
#r2 = np.array(annotations_winners(5, '../models', '../instances',5, 10))

In [ ]:
print(matrix)
# sum the rows
row_sums = np.sum(matrix, axis=0)

# print the row sums
print(row_sums)

In [ ]:
import matplotlib.pyplot as plt 

# Define figure size
fig, ax = plt.subplots(figsize=(8, 13))
# CUT
limit = 20000

models_folder = '../models'  
models = [os.path.join(models_folder, f) for f in os.listdir(models_folder) if f.endswith('.mzn')]
# Filter models and times
models = [model for i, model in enumerate(models) if times[i] < limit] 
models = [model.replace("../models\\", "").replace(".mzn", "") for model in models]
sav_times = [int(time) for time in times if time < limit]

# Use barh function to create horizontal bar chart
ax.barh(models, sav_times)

# Set x-axis label and y-axis label
ax.set_xlabel('Time (s)')
ax.set_ylabel('Models')

# Add value labels next to each bar
for i, v in enumerate(sav_times):
    ax.text(v, i, str(v), color='black', ha='left', va='center')

# Show the plot
plt.show()


## Ranking winners

In [ ]:

models_folder = '../models/tie'  
models = [os.path.join(models_folder, f) for f in os.listdir(models_folder) if f.endswith('.mzn')]

instances_folder = '../instances'
instances = [os.path.join(instances_folder, f) for f in os.listdir(instances_folder) if f.endswith('.dzn')][:10]
# Remember, you are missing the stastic sample, calculate this number using len(instances) 
times = [0] * len(models)

punishment = 100000
timeout = 25
timeout_mzn = 20*1000

for i, model in enumerate(models):
    for instance in instances:
        # Run the minizinc command for the current model and data file  
        cmd = f"minizinc --solver Gecode --output-time {model} {instance} --output-time --solver-time-limit {timeout_mzn}" 
        proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
        try: 
            # Get output from fzn
            stdout, stderr = proc.communicate(timeout=timeout)
            
            try:
                mnt = int(stdout.decode().split(' ')[-5][:-4])
            except:
                mnt = punishment
            elapsed_time = float(stdout.decode().split(' ')[-2])
            
            print_green(f'{model} with {instance} SOLVED with a value of {mnt} in {elapsed_time:.2f} seconds.')
            times[i] = (times[i]+mnt)
        except subprocess.TimeoutExpired: 
            print_red(f'Error from minizinc: Stopping.')
            times[i] = (times[i]+timeout)
            if os.name == 'nt': # If the os is Windows
                subprocess.call(['taskkill', '/F', '/T', '/PID', str(proc.pid)]) # Force kill the process
            else:
                os.killpg(os.getpgid(proc.pid), signal.SIGTERM) # Kill the process group
print("----FINISHED----")


In [ ]:
# Define figure size
fig, ax = plt.subplots(figsize=(8, 13))
# CUT
limit = 20000

models_folder = '../models/tie'  
models = [os.path.join(models_folder, f) for f in os.listdir(models_folder) if f.endswith('.mzn')]
# Filter models and times
models = [model for i, model in enumerate(models) if times[i] < limit] 
models = [model.replace("../models/tie\\", "").replace(".mzn", "") for model in models]
sav_times = [int(time) for time in times if time < limit]

# Use barh function to create horizontal bar chart
ax.barh(models, sav_times)

# Set x-axis label and y-axis label
ax.set_xlabel('Time (s)')
ax.set_ylabel('Models')

# Add value labels next to each bar
for i, v in enumerate(sav_times):
    ax.text(v, i, str(v), color='black', ha='left', va='center')

# Show the plot
plt.show()